# LEARNING PYTORCH WITH EXAMPLES

- 원본 : [링크](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)
- Author : [Justin Johnson](https://github.com/jcjohnson/pytorch-examples)
- 번역 & 주석 추가 : [박범진](https://github.com/pbj0812)

이 튜토리얼은 예를 통해 PyTorch의 컨셉을 소개한다.

중요한 점은, PyTorch는 두 가지 특징이 있다.:

- n차원의 Tensor, numpy 모듈과 비슷하지만 GPU에서 돌아간다.
- 딥러닝 학습과 설계를 위한 자동 미분

우리는 완전연결된 ReLU 네트워크를 우리의 예를 통해 사용할 것이다. 이 네트워크는 단일 은닉층을 가지고 있고, gradient descent 방법을 통해 무작위 데이터를 사용한 실제값과 네트워크 사이의 Euclidean 거리의 최소화 방법을 통해 학습될 것이다. 

## Tensors

## Warm-up: numpy

Pytorch를 소개하기 전에, 우리는 먼저 numpy를 사용하여 첫번째 시도를 진행할 것이다.

Numpy는 n차원의 행렬을 제공하며, 이 행렬들을 다루는 많은 함수들도 제공한다.<br>
Numpy는 컴퓨팅 과학을 위한 포괄적인 프레임워크이다; 이것은 computing graph, 딥러닝, gradients에 대해 전혀 알지 못한다. 하지만 우리는 Numpy 연산을 통해 랜덤한 데이터를 사용하는 2층 네트워크의 forward와 backward를 쉽게 구현할 수 있다.

In [75]:
# -*- coding: utf-8 -*-
import numpy as np

# N : batch size
# D_in : input dimension
# H : hidden dimension
# D_out : output dimension
N, D_in, H, D_out = 64, 1000, 100, 10

# 랜덤한 입력값과 아웃풋값의 생성
x = np.random.randn(N, D_in) # 64 * 1000 행렬 생성
y = np.random.randn(N, D_out) # 64 * 10 행렬 생성 

# 무작위 weights 초기화
w1 = np.random.randn(D_in, H) # 1000 * 100 행렬 생성
w2 = np.random.randn(H, D_out) # 100 * 10 행렬 생성

learning_rate = 1e-6

In [76]:
for t in range(500):
    # Forward pass : y값 예측
    h = x.dot(w1) # 64 * 100 행렬 생성
    h_relu = np.maximum(h, 0) # (64 * 100), 0 미만의 값은 0으로 반환
    y_pred = h_relu.dot(w2) # 64 * 10 행렬 생성
    
    # loss 계산
    loss = np.square(y_pred - y).sum() # 각 행렬의 원소마다의 loss 계산 및 총합 연산
    print(t, loss)
    # w1과 w2의 gradients 연산을 위한 loss 값의 Backprop
    grad_y_pred = 2.0 * (y_pred - y) # 64 * 10행렬 생성
    grad_w2 = h_relu.T.dot(grad_y_pred) # 100 * 10 행렬 생성
    grad_h_relu = grad_y_pred.dot(w2.T) # 64 * 100 행렬 생성
    grad_h = grad_h_relu.copy() # 64 * 100 행렬 생성
    grad_h[h < 0] = 0 # (64 * 100), h자리가 0 미만인 지역은 0으로 반환
    grad_w1 = x.T.dot(grad_h) # 1000 * 100 행렬 생성
    
    # Update weights
    w1 -= learning_rate * grad_w1 # 1000 * 100 행렬 생성
    w2 -= learning_rate * grad_w2 # 100 * 10 행렬 생성

0 24069628.996641867
1 18487066.20960813
2 16577518.178110804
3 15982012.624907777
4 15335441.864505116
5 13979425.463570135
6 11746139.357694829
7 9107389.288126567
8 6557273.12195369
9 4512946.239012886
10 3031387.1412338223
11 2043521.0195580246
12 1404877.8594374931
13 998777.1823023907
14 737528.1027497896
15 566283.8251230912
16 450153.1478786414
17 368386.61808493925
18 308506.48718925763
19 262952.43779533065
20 227104.52480531821
21 198082.82777227304
22 174075.5800775272
23 153881.90220097385
24 136646.0176234729
25 121789.24689139273
26 108882.56220628637
27 97601.37499711843
28 87686.31661098321
29 78951.79261843592
30 71213.71346080766
31 64337.12513296786
32 58215.425334888954
33 52757.90983448128
34 47878.2977881292
35 43505.54468365016
36 39581.39397197345
37 36053.62045342981
38 32874.50070123798
39 30007.41944799839
40 27418.30055849068
41 25077.95242207952
42 22960.91573713415
43 21041.168691968644
44 19298.268773178963
45 17717.403620763387
46 16281.205244231063
47 

424 5.5594329529867366e-05
425 5.312870905317496e-05
426 5.0773036506882e-05
427 4.852165789319217e-05
428 4.637082501929823e-05
429 4.431527127203569e-05
430 4.235132273908034e-05
431 4.047463658081106e-05
432 3.868178020728443e-05
433 3.696804799186564e-05
434 3.5330477147269126e-05
435 3.3765686919450865e-05
436 3.226995895991354e-05
437 3.084099063869241e-05
438 2.947518212374316e-05
439 2.8170172023507482e-05
440 2.692294225822732e-05
441 2.573150590249166e-05
442 2.4592608911146593e-05
443 2.35041156810157e-05
444 2.2464047895568954e-05
445 2.1469943720126175e-05
446 2.052027090464873e-05
447 1.9612415829752733e-05
448 1.874482043277338e-05
449 1.7915745353060058e-05
450 1.7123586913454454e-05
451 1.6366423361284766e-05
452 1.5642681106479705e-05
453 1.495111453011032e-05
454 1.4290112883258907e-05
455 1.3658413205887145e-05
456 1.3054664655149577e-05
457 1.2477640346900992e-05
458 1.1926343150722509e-05
459 1.1399444331659962e-05
460 1.0895820726849456e-05
461 1.0414354165678026